In [2]:
!pip install ktrain

     |################################| 25.3 MB 23.7 MB/s eta 0:00:01
     |################################| 22.2 MB 416 kB/s  eta 0:00:01
     |################################| 11.5 MB 13.1 MB/s eta 0:00:01
     |################################| 9.5 MB 18.4 MB/s eta 0:00:01
     |################################| 981 kB 41.4 MB/s eta 0:00:01
     |################################| 19.2 MB 38.3 MB/s eta 0:00:01
     |################################| 263 kB 33.4 MB/s eta 0:00:01
     |################################| 2.8 MB 34.9 MB/s eta 0:00:01
     |################################| 468 kB 36.5 MB/s eta 0:00:01
     |################################| 25.9 MB 21.5 MB/s eta 0:00:01     |##############################  | 24.9 MB 21.5 MB/s eta 0:00:01
     |################################| 1.1 MB 34.7 MB/s eta 0:00:01
     |################################| 3.1 MB 24.9 MB/s eta 0:00:01
     |################################| 503 kB 31.2 MB/s eta 0:00:01
     |#######################

In [2]:
# Influenced by this video - https://www.youtube.com/watch?v=s3LBdmZb00g

In [7]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline
import os
os.environ['CUDA_DEVICE_ORDER'] = 'PCI_BUS_ID'
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

In [8]:
import ktrain
from ktrain import text
from sklearn.datasets import fetch_20newsgroups
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 5595228507583691414
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 4273128550143583283
physical_device_desc: "device: XLA_CPU device"
, name: "/device:XLA_GPU:0"
device_type: "XLA_GPU"
memory_limit: 17179869184
locality {
}
incarnation: 5786302697338799328
physical_device_desc: "device: XLA_GPU device"
]


In [9]:
categories = ['alt.atheism','soc.religion.christian','comp.graphics','sci.med','rec.sport.baseball']

In [10]:
train = fetch_20newsgroups(subset ='train',categories=categories,shuffle=True,random_state=0)

In [11]:
test = fetch_20newsgroups(subset ='test',categories=categories,shuffle=True,random_state=0)

In [12]:
train.keys()

dict_keys(['data', 'filenames', 'target_names', 'target', 'DESCR'])

In [13]:
test.target_names

['alt.atheism',
 'comp.graphics',
 'rec.sport.baseball',
 'sci.med',
 'soc.religion.christian']

In [14]:
X_train = train.data
y_train = train.target
X_test = test.data
y_test = test.target

In [15]:
len(X_train), len(X_test)

(2854, 1899)

In [16]:
# Build the model
model_name = 'distilbert-base-uncased'
trans = text.Transformer(model_name=model_name, maxlen=512,class_names=categories)

In [17]:
train_data = trans.preprocess_train(X_train,y_train)
test_data = trans.preprocess_test(X_test, y_test)

preprocessing train...
language: en
train sequence lengths:
	mean : 291
	95percentile : 820
	99percentile : 1757


Is Multi-Label? False
preprocessing test...
language: en
test sequence lengths:
	mean : 323
	95percentile : 894
	99percentile : 2394


In [18]:
model = trans.get_classifier()

In [19]:
learner = ktrain.get_learner(model,train_data=train_data,val_data=test_data,batch_size=16)

In [27]:
#learner.lr_find(show_plot=True,max_epochs=10)

simulating training for different learning rates... this may take a few moments...
Epoch 1/10
  6/178 [>.............................] - ETA: 30:28 - loss: 1.6093 - accuracy: 0.1979

In [ ]:
learner.fit_onecycle(1e-4,1)



begin training using onecycle policy with max lr of 0.0001...
 36/179 [=====>........................] - ETA: 39:12 - loss: 1.3398 - accuracy: 0.5590

In [ ]:
learner

In [ ]:
learner.validate()

In [ ]:
learner.validate(class_names=categories)

In [ ]:
learner.view_top_losses(n=5,preproc=trans)

In [ ]:
#Predict on new data
predictor = ktrain.get_predictor(learner.model, prepoc=trans)
x = 'Baseball is the best game that one can learn and play.'

predictor.predict(x)

In [ ]:
predictor.save("model_news_distilbert")